In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
import os
import sys
import pyproj

from if_license_plates_could_talk import data
from if_license_plates_could_talk import geo

In [8]:
df_geo = geo.utils.load_geodata()
df = data.db.get_data()

In [9]:
df.head()

,kreis_key,kreis_name,income_pp_2000,income_pp_2001,income_pp_2002,income_pp_2003,income_pp_2004,income_pp_2005,income_pp_2006,income_pp_2007,...,prop_no_haupt_2011,prop_no_haupt_2012,prop_no_haupt_2013,prop_no_haupt_2014,prop_no_haupt_2015,prop_no_haupt_2016,prop_no_haupt_2017,prop_no_haupt_2018,prop_no_haupt_2019,avg_hh
0,09361,Amberg,16120.0,16451.0,16433.0,16878.0,17318.0,17724.0,18326.0,18841.0,...,5.2,5.0,6.8,3.7,4.8,5.8,6.2,5.7,5.6,2.017355
1,09561,Ansbach,17507.0,18162.0,17736.0,17849.0,18038.0,18022.0,17955.0,18110.0,...,1.9,2.8,2.9,1.1,3.3,2.9,6.0,4.2,3.4,2.045823
2,09661,Aschaffenburg,17228.0,17972.0,17975.0,18703.0,19305.0,19802.0,20393.0,20802.0,...,3.7,3.7,3.8,4.4,5.5,8.3,3.8,5.7,5.0,2.043977
3,09761,Augsburg,15590.0,16010.0,15969.0,16279.0,16471.0,16432.0,16547.0,16754.0,...,3.6,4.4,4.6,3.9,4.5,4.9,6.8,5.8,6.1,1.913406
4,08211,Baden-Baden,25133.0,25229.0,23987.0,23945.0,23923.0,25003.0,26262.0,27180.0,...,3.8,3.3,4.9,4.1,4.5,7.7,4.8,6.2,7.3,1.965715


In [10]:
df.head()

,kreis_key,kreis_name,income_pp_2000,income_pp_2001,income_pp_2002,income_pp_2003,income_pp_2004,income_pp_2005,income_pp_2006,income_pp_2007,...,prop_no_haupt_2011,prop_no_haupt_2012,prop_no_haupt_2013,prop_no_haupt_2014,prop_no_haupt_2015,prop_no_haupt_2016,prop_no_haupt_2017,prop_no_haupt_2018,prop_no_haupt_2019,avg_hh
0,09361,Amberg,16120.0,16451.0,16433.0,16878.0,17318.0,17724.0,18326.0,18841.0,...,5.2,5.0,6.8,3.7,4.8,5.8,6.2,5.7,5.6,2.017355
1,09561,Ansbach,17507.0,18162.0,17736.0,17849.0,18038.0,18022.0,17955.0,18110.0,...,1.9,2.8,2.9,1.1,3.3,2.9,6.0,4.2,3.4,2.045823
2,09661,Aschaffenburg,17228.0,17972.0,17975.0,18703.0,19305.0,19802.0,20393.0,20802.0,...,3.7,3.7,3.8,4.4,5.5,8.3,3.8,5.7,5.0,2.043977
3,09761,Augsburg,15590.0,16010.0,15969.0,16279.0,16471.0,16432.0,16547.0,16754.0,...,3.6,4.4,4.6,3.9,4.5,4.9,6.8,5.8,6.1,1.913406
4,08211,Baden-Baden,25133.0,25229.0,23987.0,23945.0,23923.0,25003.0,26262.0,27180.0,...,3.8,3.3,4.9,4.1,4.5,7.7,4.8,6.2,7.3,1.965715


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 0 to 472
Columns: 115 entries, kreis_key to avg_hh
dtypes: float64(113), object(2)
memory usage: 428.7+ KB


In [12]:
df_comb = df_geo.merge(df, left_on = "RS", right_on = "kreis_key", how = "left")

In [13]:
feature_info = {
            "crimes_pp": {
                "title": "Erfasste Straftaten",
                "label": "Straftaten / EW"
            },
            "income_pp": {
                "title": "Verfügbares Einkommen der priv. Haushalte",
                "label": "Euro / EW"
            },
            "population": {
                "title": "Bevölkerung",
                "label": "EW"
            }
}

In [14]:
def generate_static_map(feature, year):
    col = f"{feature}_{year}"
    title = feature_info[feature]["title"]
    fig = px.choropleth(df_comb, geojson=df_comb.geometry, locations=df_comb.index, color=col, scope="europe",
                            color_continuous_scale="gray_r",
                            title=f"{title} ({year})",
                            range_color=(df_comb[col].min(
                            )*0.8, df[col].max()),
                            labels={
                                col: feature_info[feature]["label"]
                            })
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(hoverlabel={"bgcolor": "white"}, title_font_color="black", font_color="black")

    fig.add_annotation(dict(font=dict(color='black',size=11),
                                        x=-0.1,
                                        y=-0.12,
                                        showarrow=False,
                                        text="Quelle: BKA, Regionaldatenbank Deutschland",
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))

    fig.write_image(os.path.join("..", "output", f"{col}.png"), scale=1)
    fig.write_html(os.path.join("..", "output", f"{col}.html"))

In [15]:
generate_static_map("income_pp", 2018)
generate_static_map("crimes_pp", 2018)
generate_static_map("population", 2018)